# Classes

In [155]:
from dataclasses import dataclass
from typing import List
import math as m

We define all of the components of our simulation as dataclass and then store them inside of the Simulation class. This allows use to create a "memory" by storing the environment inside of previous states everytime we modify it.

In [156]:
# Earth's radius in meters
EARTH_RADIUS = 6371000

def great_circle_distance(latitude1: float, longitude1: float, latitude2: float, longitude2: float, radius: float) -> float:
    """Compute the greater circle distance on a sphere using the Haversine formula"""
    
    phi1 = m.radians(latitude1)
    phi2 = m.radians(latitude2)

    delta_phi = m.radians(latitude2 - latitude1)
    delta_lambda = m.radians(longitude2 - longitude1)

    a = m.sin(delta_phi / 2.0) ** 2 + m.cos(phi1) * m.cos(phi2) * m.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * m.atan2(m.sqrt(a), m.sqrt(1 - a))

    meters = radius * c
    return meters

def distance_with_altitude(distance: float, altitude1: float, altitude2: float):
    return m.sqrt(distance ** 2 + (altitude2 - altitude1) ** 2)


In [157]:
from __future__ import annotations

class Radians:
    def convert_from_degrees(degrees: float):
        return degrees * m.pi / 180
    
    def convert_to_degrees(radians: float):
        return radians * 180 / m.pi

@dataclass(slots=True)
class Location:
    """Represents a location on a sphere."""
    
    latitude: float # in degrees
    longitude: float # in degrees
    altitude: float # in meters
    
    def distance(a: Location, b: Location, radius: float = EARTH_RADIUS) -> float:
        d = great_circle_distance(a.latitude, a.longitude, b.latitude, b.longitude, radius)
        return distance_with_altitude(d, a.altitude, b.altitude)
    
    def bearing(a: Location, b: Location) -> float:
        latitude_a_radians = Radians.convert_from_degrees(a.latitude)
        latitude_b_radians = Radians.convert_from_degrees(b.latitude)
        longitude_delta_radians = Radians.convert_from_degrees(b.longitude - a.longitude)
        
        y = m.sin(longitude_delta_radians) * m.cos(latitude_b_radians)
        x = m.cos(latitude_a_radians) * m.sin(latitude_b_radians) - m.sin(latitude_a_radians) * m.cos(latitude_b_radians) * m.cos(longitude_delta_radians)
        bearing_radians = m.atan2(y, x)
        
        return (Radians.convert_to_degrees(bearing_radians) + 360) % 360

    
    def interpolated_position_towards_target(a: Location, b: Location, distance: float, radius: float = EARTH_RADIUS) -> Location:        
        """Interpolation a position on a sphere of a given radius that is distance towards b from a"""

        distance_radians = distance / radius
        bearing_radians = Radians.convert_from_degrees(Location.bearing(a, b))
        
        initial_latitude_radians = Radians.convert_from_degrees(a.latitude)
        initial_longitude_radians = Radians.convert_from_degrees(a.longitude)
        
        destination_latitude_radians = m.asin(m.sin(initial_latitude_radians) * m.cos(distance_radians) + m.cos(initial_latitude_radians) * m.sin(distance_radians) * m.cos(bearing_radians))
        destination_longitude_radians = initial_longitude_radians + m.atan2(m.sin(bearing_radians) * m.sin(distance_radians) * m.cos(initial_latitude_radians), m.cos(distance_radians) - m.sin(initial_latitude_radians) * m.sin(destination_latitude_radians))        
        
        # Normalize destination longitude to be within -pi and +pi radians
        destination_longitude_radians = (destination_longitude_radians + 3 * m.pi) % (2 * m.pi) - m.pi
    
        percentage_of_distance_traveled = distance / Location.distance(a, b)
        destination_altitude = a.altitude + percentage_of_distance_traveled * (b.altitude - a.altitude)
    
        return Location(Radians.convert_to_degrees(destination_latitude_radians), Radians.convert_to_degrees(destination_longitude_radians), destination_altitude)
    
@dataclass(slots=True)
class Motor:
    rpm: int
    
    def ramp_to_target(self, rpm: int, time_delta: int) -> None:
        # TODO: Implement ramping
        self.rpm = rpm

@dataclass(slots=True)
class Car:
    location: Location
    motor: Motor
    throttle: float
    
    def get_distance_traveled(self, time_delta: int) -> float:
        """Computes how much distance would be traveled in time_delta. Important to note that this does not move the car."""
        return time_delta / 1000 # Hardcoded 1 m/s
    
    def move_to(self, location: Location):
        self.location = location
    
@dataclass(slots=True)
class Environment:
    time: int
    car: Car
    waypoints: List[Location]
    
    def get_distance_to_next_waypoint(self) -> float:
        """Distance to the first waypoint in waypoints. Returns infinity if there are no waypoints left."""
        
        if len(self.waypoints) == 0:
            return m.inf
        
        return Location.distance(self.car.location, self.waypoints[0])
    
    def move_car_towards_target(self, time_delta: int):
        if len(self.waypoints) == 0:
            print("No waypoint to move towards")
            return
        
        distance = self.car.get_distance_traveled(time_delta)
        new_position = Location.interpolated_position_towards_target(self.car.location, self.waypoints[0], distance)
        
        self.car.move_to(new_position)
        
    def __repr__(self) -> str:
        return f"time {self.time}:\n{self.car}\nNext three waypoints:\n{self.waypoints[:3]}"
    
class Simulation:
    saved_states: List[Environment]
    environment: Environment
    
    def __init__(self, waypoints: List[Location]) -> None:
        if len(waypoints) == 0:
            raise ValueError("Cannot construct simulation without waypoints")
        
        self.saved_states = []

        car = Car(location=waypoints[0], motor=Motor(0), throttle=0)
        self.environment = Environment(time=0, car=car, waypoints=waypoints[1:])

    def step(self, time_delta: int, time_step_size = 1, waypoint_distance_threshold = 1) -> None:    
        if time_step_size > time_delta:
            raise ValueError("Time Step Size cannot be smaller than the Time Delta")
        
        if len(self.environment.waypoints) == 0:
            print("Skipping step. No waypoints left")
            self.time += time_delta
            return
        
        step_time = 0
        while step_time < time_delta:    
            self.environment.move_car_towards_target(time_step_size)

            if self.environment.get_distance_to_next_waypoint() < waypoint_distance_threshold:
                waypoint = self.environment.waypoints.pop(0)
                print(f"Traversed {waypoint}")
           
            step_time += time_step_size

        self.environment.time += step_time
        self.saved_states.append(self.environment)
        
    def __repr__(self) -> str:
        return f"Simulation:\n{self.environment}"

In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as m

track_data = pd.read_csv("./sem_2023_us.csv")

track_data = track_data.rename(columns={
    "Metres above sea level": "Altitude"
})

track_data.head(10)

,Latitude,Longitude,Altitude
0,39.799168,-86.238014,222.3313
1,39.799173,-86.237999,222.3617
2,39.799179,-86.237985,222.3676
3,39.799186,-86.237972,222.3918
4,39.799193,-86.237960,222.3986
5,39.799200,-86.237946,222.4268
6,39.799207,-86.237932,222.4488
7,39.799214,-86.237918,222.4692
8,39.799221,-86.237905,222.4826
9,39.799228,-86.237890,222.5074


In [159]:
track_data["PreviousLatitude"] = track_data["Latitude"].shift(1)
track_data["PreviousLongitude"] = track_data["Longitude"].shift(1)
track_data["PreviousAltitude"] = track_data["Altitude"].shift(1)

track_data["FlatDistance"] = track_data.apply(lambda row: great_circle_distance(row["Latitude"], row["Longitude"], row["PreviousLatitude"], row["PreviousLongitude"], EARTH_RADIUS), axis=1)
track_data["AltitudeDistance"] = track_data.apply(lambda row: row["PreviousAltitude"] - row["Altitude"], axis=1)
track_data["Distance"] = track_data.apply(lambda row: distance_with_altitude(row["FlatDistance"], row["Altitude"], row["PreviousAltitude"]), axis=1)

track_data.head(10)


,Latitude,Longitude,Altitude,PreviousLatitude,PreviousLongitude,PreviousAltitude,FlatDistance,AltitudeDistance,Distance
0,39.799168,-86.238014,222.3313,NaN,NaN,NaN,NaN,NaN,NaN
1,39.799173,-86.237999,222.3617,39.799168,-86.238014,222.3313,1.445678,-0.0304,1.445997
2,39.799179,-86.237985,222.3676,39.799173,-86.237999,222.3617,1.375121,-0.0059,1.375133
3,39.799186,-86.237972,222.3918,39.799179,-86.237985,222.3676,1.334003,-0.0242,1.334222
4,39.799193,-86.237960,222.3986,39.799186,-86.237972,222.3918,1.307455,-0.0068,1.307473
5,39.799200,-86.237946,222.4268,39.799193,-86.237960,222.3986,1.386252,-0.0282,1.386539
6,39.799207,-86.237932,222.4488,39.799200,-86.237946,222.4268,1.433784,-0.0220,1.433953
7,39.799214,-86.237918,222.4692,39.799207,-86.237932,222.4488,1.422830,-0.0204,1.422976
8,39.799221,-86.237905,222.4826,39.799214,-86.237918,222.4692,1.386886,-0.0134,1.386951
9,39.799228,-86.237890,222.5074,39.799221,-86.237905,222.4826,1.437821,-0.0248,1.438035


In [160]:
waypoints = []
for index, row in track_data.iterrows():
    waypoint = Location(latitude=row["Latitude"], longitude=row["Longitude"], altitude=row["Altitude"])
    waypoints.append(waypoint)
    
print(waypoints[:10])

[Location(latitude=39.799168014, longitude=-86.23801415599999, altitude=222.3313), Location(latitude=39.799173325, longitude=-86.23799871, altitude=222.3617), Location(latitude=39.799179425, longitude=-86.237984708, altitude=222.3676), Location(latitude=39.799186418, longitude=-86.23797202, altitude=222.3918), Location(latitude=39.799193315, longitude=-86.237959625, altitude=222.3986), Location(latitude=39.799200062, longitude=-86.23794597999999, altitude=222.4268), Location(latitude=39.799207064, longitude=-86.237931887, altitude=222.4488), Location(latitude=39.799214173, longitude=-86.237918039, altitude=222.46920000000003), Location(latitude=39.799221068, longitude=-86.237904511, altitude=222.4826), Location(latitude=39.799228139, longitude=-86.23789042, altitude=222.5074)]


In [161]:
sim = Simulation(waypoints)


print(f"Initial State:\n{sim}")

while len(sim.environment.waypoints) != 0:
    sim.step(10)
    
print("")
    
print("End:")
print(sim.saved_states[len(sim.saved_states) - 1])

Initial State:
Simulation:
time 0:
Car(location=Location(latitude=39.799168014, longitude=-86.23801415599999, altitude=222.3313), motor=Motor(rpm=0), throttle=0)
Next three waypoints:
[Location(latitude=39.799173325, longitude=-86.23799871, altitude=222.3617), Location(latitude=39.799179425, longitude=-86.237984708, altitude=222.3676), Location(latitude=39.799186418, longitude=-86.23797202, altitude=222.3918)]
Traversed Location(latitude=39.799173325, longitude=-86.23799871, altitude=222.3617)
Traversed Location(latitude=39.799179425, longitude=-86.237984708, altitude=222.3676)
Traversed Location(latitude=39.799186418, longitude=-86.23797202, altitude=222.3918)
Traversed Location(latitude=39.799193315, longitude=-86.237959625, altitude=222.3986)
Traversed Location(latitude=39.799200062, longitude=-86.23794597999999, altitude=222.4268)
Traversed Location(latitude=39.799207064, longitude=-86.237931887, altitude=222.4488)
Traversed Location(latitude=39.799214173, longitude=-86.237918039, 